In [1]:
#Python Packages.
import os
import time
import random
import numpy as np

# Torch Packages
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim
import torch.utils.data
import torchvision.transforms as transforms
import torchvision.datasets as datasets
#import torchvision.models as models
import torch.backends.cudnn as cudnn

# Import the Model.
#from model import image_ingredient
from decoder import HierRNN
#from data_loader import ImageLoader
import pdb
# Packages which will help Loading the data.
import pickle
#import lmdb
from PIL import Image


# Give the path 
IMG_PATH = '/home/yifu/Documents/Mycode/python/hierarchicalRNN/jasha/'
# GIve the path for the LMDB files that were created.
DATA_PATH = '/home/yifu/Documents/Mycode/python/hierarchicalRNN/jasha/lmdb/'
WORKERS = 0
BATCH_SIZE = 60
VAL_FREQ = 1
START_EPOCH = 0
TOTAL_EPOCH = 500


#batch_size = 50 # Being support batch_size
num_boxes = 50 # number of Detected regions in each image
feats_dim = 4096 # feature dimensions of each regions
project_dim = 1024 # project the features to one vector, which is 1024 dimensions

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words

S_max = 3
N_max = 20
T_stop = 0.5


#500
n_epochs = 500
learning_rate = 0.0001

def getIdx():
    ixtoword = {}
    wordtoix = {}
    ingr_vocab = {}
    #Vocab.text is the pre-processed corpus of all the instructions.
    with open('../files/vocab.txt') as f_vocab:
        #wordtoix a dict mapped to the index values. Key is the word and value is the index value.
        #ingr_vocab = {w.rstrip(): i+2 for i, w in enumerate(f_vocab)} # +1 for lua
        #ingr_vocab['</i>'] = 1
        #print (ingr_vocab)
        for i, w in enumerate(f_vocab):
            word = w.rstrip()
            ixtoword[i+5] = word
            wordtoix[w.rstrip()] = i+5
        wordtoix['</i>'] = 4
        wordtoix['<bos>'] = 0
        wordtoix['<eos>'] = 1
        wordtoix['<pad>'] = 2
        wordtoix['<unk>'] = 3
        ixtoword[0] = '<bos>'
        ixtoword[1] = '<eos>'
        ixtoword[2] = '<pad>'
        ixtoword[3] = '<unk>'
        ixtoword[4] = '</i>'
    return wordtoix

sentRNN_lstm_dim = 512 # the sentence LSTM hidden units
sentRNN_FC_dim = 1024 # the fully connected units
wordRNN_lstm_dim = 512 # the word LSTM hidden units
word_embed_dim = 1024 # the learned embedding vectors for the words


word2idx = getIdx()
model = HierRNN(n_words = len(word2idx),
                                          batch_size = BATCH_SIZE,
                                          num_boxes = num_boxes,
                                          feats_dim = feats_dim,
                                          project_dim = project_dim,
                                          sentRNN_lstm_dim = sentRNN_lstm_dim,
                                          sentRNN_FC_dim = sentRNN_FC_dim,
                                          wordRNN_lstm_dim = wordRNN_lstm_dim,
                                          S_max = S_max,
                                          N_max = N_max,
                                          word_embed_dim = word_embed_dim
                                            #bias_init_vector = bias_init_vector
                     )

In [2]:
feature = torch.randn([10,1024])

In [4]:
model.load_state_dict(torch.load('snapshots/model_e001_v-01.pth.tar')['decoder_state_dict'])

KeyError: 'decoder_state_dict'

In [33]:
x = torch.load('snapshots/model_e001_v-01.pth.tar')

In [35]:
x["state_dict"]

OrderedDict([('visionMLP.0.weight',
              tensor([[[[ 5.5990e-03,  1.4756e-02, -1.5258e-02,  ..., -4.1431e-02,
                         -4.3344e-02, -7.1064e-02],
                        [ 8.4448e-03,  4.1022e-02,  5.0122e-02,  ...,  2.4658e-02,
                          1.0016e-02, -7.5890e-03],
                        [ 2.6655e-02,  5.8769e-02,  5.1294e-02,  ...,  1.2526e-01,
                          9.3570e-02,  8.2905e-02],
                        ...,
                        [ 3.4777e-03,  1.1865e-01,  8.0775e-02,  ...,  1.6371e-02,
                          3.8807e-02,  1.2326e-01],
                        [ 7.8920e-03,  8.0744e-02,  9.4790e-02,  ...,  4.3515e-02,
                         -4.8660e-03,  1.3087e-02],
                        [-7.5727e-02,  5.0015e-04,  1.4931e-02,  ...,  5.4607e-02,
                          5.1235e-02,  3.0504e-02]],
              
                       [[-2.6183e-02,  1.1517e-02,  2.3952e-02,  ...,  5.3210e-02,
                          